### Heroes Of Pymoli Data Analysis

* The 35-39 age group has a usd 4.76 Avg Total Purchase per Person, which is the highest of all group ages by a significant margin. E.g. 20-24, the largest age group, has an Avg Total Purchase per Person of usd 4.32.

* Female players have a higher Total Purchase per Person than males (usd 4.47 vs usd 4.07). Although the source data only contains 113 female players which may not be enough to securely identify a trend.

* It may be advisable to customize the game to attract more players within the demographics: age 35-39, and Female.

* Items with an Item ID of 178, 145, 108, 82 and 19 are the top sellers. It may be advisable to understand what features these items have in common and try to replicate it them in other items.

-----

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)

## Player Count

* Display the total number of players


In [2]:
Total_Players = len(purchase_data["SN"].unique())
Total_Players_df = pd.DataFrame({"Total Players": [Total_Players]})
Total_Players_df

,Total Players
0,576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [3]:
Number_of_Unique_Items = len(purchase_data["Item ID"].unique())
Number_of_Purchases = len(purchase_data["SN"])
#pd.DataFrame({"Total Players": [Total_Players]})
#purchase_data.head(1)
#Total_Item_ID = len(purchase_data["Item ID"].unique())
average_price = purchase_data["Price"].mean()
total_revenue = purchase_data["Price"].sum()
Purchasing_Analysis_Total_df = pd.DataFrame({
                 "Number of Unique Items" : [Number_of_Unique_Items],
                 "Average Price": [average_price],
                 "Number of Purchases": [Number_of_Purchases],
                 "total revenue": [total_revenue]
                 })

Purchasing_Analysis_Total_df["Average Price"] = Purchasing_Analysis_Total_df["Average Price"].apply('${:.2f}%'.format)
Purchasing_Analysis_Total_df["total revenue"] = Purchasing_Analysis_Total_df["total revenue"].apply('${:,.2f}'.format)

Purchasing_Analysis_Total_df


,Number of Unique Items,Average Price,Number of Purchases,total revenue
0,183,$3.05%,780,"$2,379.77"


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [4]:
purchase_data_unique_SN = purchase_data.drop_duplicates("SN")
Number_of_Purchases = len(purchase_data_unique_SN["SN"])
Number_of_Purchases

male_count = (purchase_data_unique_SN.Gender == 'Male').sum()
female_count = (purchase_data_unique_SN.Gender == 'Female').sum()
other_count = (purchase_data_unique_SN.Gender == 'Other / Non-Disclosed').sum()

Gender_Demographics_df = pd.DataFrame(
    {"Total Count": [male_count, female_count, other_count],
     "Percentage of Players" : [(male_count*100/Number_of_Purchases),(female_count*100/Number_of_Purchases),(other_count*100/Number_of_Purchases)],
    "Gender": ["Male", "Female", "Other"]}
                     )
Gender_Demographics_df = Gender_Demographics_df.set_index("Gender")
Gender_Demographics_df["Percentage of Players"] = Gender_Demographics_df["Percentage of Players"].apply('{:,.2f}%'.format)
Gender_Demographics_df


,Total Count,Percentage of Players
Gender,,
Male,484,84.03%
Female,81,14.06%
Other,11,1.91%



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [5]:
grouped_gender_df = purchase_data.groupby("Gender")
gender_summary = grouped_gender_df.count()
gender_summary_pd = pd.DataFrame(gender_summary)
gender_summary_pd

gender_sum = grouped_gender_df["Price"].sum()
gender_sum_df = pd.DataFrame(gender_sum)

renamed_df = gender_summary_pd

#use iloc to select only first column 'Pruchase Count'
a = renamed_df.iloc[:,0:1]

b = grouped_gender_df["Price"].mean()
b = pd.DataFrame(b)

c = pd.merge(a, b, on="Gender")


#calculate Avg Total Purchase per Person
Avg_Total_Purchase_per_Female = grouped_gender_df["Price"].sum()/female_count
female_no_iloc_df = pd.DataFrame(Avg_Total_Purchase_per_Female)
female_row_df = female_no_iloc_df.iloc[0]
Avg_Total_Purchase_per_Male = grouped_gender_df["Price"].sum()/male_count
male_no_iloc_df = pd.DataFrame(Avg_Total_Purchase_per_Male)
male_row_df = male_no_iloc_df.iloc[1]
Avg_Total_Purchase_per_Other = grouped_gender_df["Price"].sum()/other_count
other_no_iloc_df = pd.DataFrame(Avg_Total_Purchase_per_Other)
other_row_df = other_no_iloc_df.iloc[2]
female_num = female_row_df['Price']
male_num = male_row_df['Price']
other_num = other_row_df['Price']
Avg_Total_Purchase_per_Person_series = pd.Series([female_num, male_num, other_num],
                                index=['Female', 'Male', 'Other / Non-Disclosed'])


Avg_Total_Purchase_per_Person_df = pd.DataFrame(Avg_Total_Purchase_per_Person_series)

Avg_Total_Purchase_per_Person_df.index.name = 'Gender'

d = pd.merge(c, Avg_Total_Purchase_per_Person_df, on="Gender")


d.rename( columns={0:'Avg Total Purchase per Person',
                   'Price':'Average Purchase Price',
                  'Purchase ID':'Purchase Count'}, inplace=True )


final_df = pd.merge(d, gender_sum_df, on="Gender")

final_df.rename( columns={'Price':'Total Purchase Value'}, inplace=True )

#add formating, $ sign and two decimal points
final_df['Avg Total Purchase per Person'] = final_df['Avg Total Purchase per Person'].apply('${:,.2f}'.format)
final_df['Total Purchase Value'] = final_df['Total Purchase Value'].apply('${:,.2f}'.format)
final_df['Average Purchase Price'] = final_df['Average Purchase Price'].apply('${:,.2f}'.format)
final_df


,Purchase Count,Average Purchase Price,Avg Total Purchase per Person,Total Purchase Value
Gender,,,,
Female,113,$3.20,$4.47,$361.94
Male,652,$3.02,$4.07,"$1,967.64"
Other / Non-Disclosed,15,$3.35,$4.56,$50.19


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [6]:
age_gender_unique_df = purchase_data_unique_SN.loc[:,["Age"]]


#aa = pd.cut(age_gender_unique_df["Age"], bins, labels=group_names)

hp_bins = [0, 9, 14, 19, 24, 29, 34, 39, 100]
hp_labels = ["<10", "10-14", "15-19", "20-24","25-29","30-34","35-30", "40+"]

# Bin the Horsepower column
# cut() returns a Pandas Series containing each of the binned column's values translated into their corresponding bins

age_gender_unique_df['age ranges'] = pd.cut(purchase_data_unique_SN["Age"], hp_bins, labels=hp_labels)
age_gender_unique_df
grouped_age_gender_unique_df = age_gender_unique_df.groupby("age ranges")
grouped_age_gender_unique_df[["Age"]].count()
first_column_df = pd.DataFrame(grouped_age_gender_unique_df[["Age"]].count())
#create ned df with second column, percentages
calute_percent_df = 100*(first_column_df)/Total_Players

df_merged = first_column_df.merge(calute_percent_df, how='outer', left_index=True, right_index=True)
df_merged

df_merged = df_merged.rename(columns={"Age_x":"Total Count", "Age_y":"Percentage of Players"})

df_merged['Percentage of Players'] = df_merged['Percentage of Players'].apply('{:,.2f}'.format)
df_merged


,Total Count,Percentage of Players
age ranges,,
<10,17,2.95
10-14,22,3.82
15-19,107,18.58
20-24,258,44.79
25-29,77,13.37
30-34,52,9.03
35-30,31,5.38
40+,12,2.08


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [7]:
hp_bins = [0, 9, 14, 19, 24, 29, 34, 39, 100]
hp_labels = ["<10", "10-14", "15-19", "20-24","25-29","30-34","35-30", "40+"]

purchase_data["age ranges"] = pd.cut(purchase_data["Age"], hp_bins, labels=hp_labels)
purchase_data

new_df = purchase_data.groupby("age ranges")
a = new_df[["Purchase ID"]].count()
b = new_df[["Price"]].mean()
c = new_df[["Price"]].sum()
a = pd.DataFrame(a)
b = pd.DataFrame(b)
c = pd.DataFrame(c)

#merging by index
a = a.merge(b, how='outer', left_index=True, right_index=True)
a = a.merge(c, how='outer', left_index=True, right_index=True)
#add column from previous calucation representing the Total Count under the name 'first_column_df'
a = a.merge(first_column_df, how='outer', left_index=True, right_index=True)
#calcualte 'Avg Total Purchase per Person' by dividing Total Purchase Value by Total Count from previous calculation
a["Age"] = a["Price_y"]/a["Age"]

a = a.rename(columns={'Purchase ID': 'Purchase Count',
                      'Price_x' : 'Average Purchase Price',
                      'Price_y' : 'Total Purchase Value',
                      'Age' : 'Avg Total Purchase per Person'})

a['Average Purchase Price'] = a['Average Purchase Price'].apply('${:,.2f}'.format)
a['Total Purchase Value'] = a['Total Purchase Value'].apply('${:,.2f}'.format)
a['Avg Total Purchase per Person'] = a['Avg Total Purchase per Person'].apply('${:,.2f}'.format)
a



,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
age ranges,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-30,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [8]:
grouped_SN_df = purchase_data.groupby("SN")

#calucalate the Purchase Count
grouped_Pruchase_Count_df = grouped_SN_df[["Purchase ID"]].count()
#calculate the Average Purchase Price
grouped_Price_Average_df = grouped_SN_df[["Price"]].mean()
#calculate the Total Purchase Value
grouped_Price_Total_Purchase_df = grouped_SN_df[["Price"]].sum()

#sort Price Total Purchase (still called Price at this stage)
grouped_Price_Total_Purchase_df = grouped_Price_Total_Purchase_df.sort_values('Price', ascending=False)
grouped_Price_Total_Purchase_df


#merging by index (note that when merging by index column names with the same, e.g. 'price' will have and _x, _y,... added)
top_spenders_df = grouped_Price_Total_Purchase_df.merge(grouped_Pruchase_Count_df, how='outer', left_index=True, right_index=True)
top_spenders_df = top_spenders_df.merge(grouped_Price_Average_df, how='outer', left_index=True, right_index=True)

#short by Total Purcahse Value (note, still called 'Price_x' at this stage)
top_spenders_df = top_spenders_df.sort_values('Price_x', ascending=False)

#renaming columns
top_spenders_df = top_spenders_df.rename(columns={'Purchase ID': 'Purchase Count',
                      'Price_x' : 'Total Purchase Value',
                      'Price_y' : 'Average Purchase Price',
                      })
#formating collumns
top_spenders_df['Average Purchase Price'] = top_spenders_df['Average Purchase Price'].apply('${:,.2f}'.format)
top_spenders_df['Total Purchase Value'] = top_spenders_df['Total Purchase Value'].apply('${:,.2f}'.format)

top_spenders_df = top_spenders_df[["Purchase Count","Average Purchase Price","Total Purchase Value"]]

top_spenders_df

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10
Ilarin91,3,$4.23,$12.70
Ialallo29,3,$3.95,$11.84
Tyidaim51,3,$3.94,$11.83
Lassilsala30,3,$3.84,$11.51


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [10]:

grouped_Item_df = purchase_data.groupby(['Item ID', 'Item Name', 'Price']).size()
grouped_Item_df = pd.DataFrame(grouped_Item_df)
grouped_Item_df = grouped_Item_df.rename(columns={0: 'Purchase Count'})

#convert Price from Index into column
grouped_Item_df = grouped_Item_df.reset_index(level=['Price'])

grouped_Item_df['Total Purchase Value'] = grouped_Item_df['Price']*grouped_Item_df['Purchase Count']

grouped_Item_df = grouped_Item_df.sort_values('Purchase Count', ascending=False)

grouped_Item_df['Price'] = grouped_Item_df['Price'].apply('${:,.2f}'.format)
grouped_Item_df['Total Purchase Value'] = grouped_Item_df['Total Purchase Value'].apply('${:,.2f}'.format)

grouped_Item_df = grouped_Item_df[["Purchase Count","Price","Total Purchase Value"]]

grouped_Item_df

,,Purchase Count,Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16
103,Singed Scalpel,8,$4.35,$34.80
75,Brutality Ivory Warmace,8,$2.42,$19.36
72,Winter's Bite,8,$3.77,$30.16
60,Wolf,8,$3.54,$28.32


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [12]:

grouped_Item_df = purchase_data.groupby(['Item ID', 'Item Name', 'Price']).size()
grouped_Item_df = pd.DataFrame(grouped_Item_df)
grouped_Item_df = grouped_Item_df.rename(columns={0: 'Purchase Count'})

#convert Price from Index into column
grouped_Item_df = grouped_Item_df.reset_index(level=['Price'])

grouped_Item_df['Total Purchase Value'] = grouped_Item_df['Price']*grouped_Item_df['Purchase Count']

grouped_Item_df = grouped_Item_df.sort_values('Total Purchase Value', ascending=False)

grouped_Item_df['Price'] = grouped_Item_df['Price'].apply('${:,.2f}'.format)
grouped_Item_df['Total Purchase Value'] = grouped_Item_df['Total Purchase Value'].apply('${:,.2f}'.format)

grouped_Item_df = grouped_Item_df[["Purchase Count","Price","Total Purchase Value"]]

grouped_Item_df

,,Purchase Count,Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80
59,"Lightning, Etcher of the King",8,$4.23,$33.84
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
78,"Glimmer, Ender of the Moon",7,$4.40,$30.80
72,Winter's Bite,8,$3.77,$30.16
